## Install deps

In [16]:
# %pip install pandas
# %pip install scikit-learn
# %pip install seaborn

## Imports

In [17]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import pandas as pd

## Load dataset

In [18]:
dataframe = read_csv("clear_all_data.csv", index_col=0)
dataframe.head()

,city,floor,floors_count,rooms_count,total_meters,price_per_m2,price,year_of_construction,living_meters,kitchen_meters,district
0,Москва,1,5,1,12.00,291666,3500000,2012,19.0,9.1,Москва
1,Москва,17,41,1,40.40,618811,25000000,2021,15.2,9.1,Москва
2,Москва,5,14,1,61.97,919799,57000000,2022,19.0,9.1,Москва
3,Москва,9,41,1,40.50,617283,25000000,2022,17.3,9.3,Москва
4,Москва,12,17,1,47.60,324579,15450000,2019,19.0,10.0,Москва


In [19]:
dataframe.head()

,city,floor,floors_count,rooms_count,total_meters,price_per_m2,price,year_of_construction,living_meters,kitchen_meters,district
0,Москва,1,5,1,12.00,291666,3500000,2012,19.0,9.1,Москва
1,Москва,17,41,1,40.40,618811,25000000,2021,15.2,9.1,Москва
2,Москва,5,14,1,61.97,919799,57000000,2022,19.0,9.1,Москва
3,Москва,9,41,1,40.50,617283,25000000,2022,17.3,9.3,Москва
4,Москва,12,17,1,47.60,324579,15450000,2019,19.0,10.0,Москва


In [20]:
# dataframe = pd.get_dummies(dataframe, columns=["district", "street", "underground"])
# dataframe.head()

## Define args and y

In [21]:
X = dataframe.drop("price_per_m2", axis=1)
y = dataframe.price_per_m2


In [22]:
X.shape

(8307, 10)

In [23]:
y.shape

(8307,)

## Split to train and test dataframes

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [25]:
from category_encoders import CatBoostEncoder

cat_features = ["city", "district"]
encoder = CatBoostEncoder()
X_train_encoded = X_train.copy()
X_train_encoded[cat_features] = encoder.fit_transform(X_train[cat_features], y_train)

In [26]:
X_test_encoded = X_test.copy()
X_test_encoded[cat_features] = encoder.transform(X_test[cat_features])

## Define and train models

In [27]:
model_list = []

### Lasso regression

* ### Create CV for lasso regression model

In [28]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=5)

* ### Learn model

In [29]:
from sklearn.linear_model import LassoCV
lasso_model = LassoCV(cv=cv, n_jobs=-1, max_iter=10000)
lasso_model.fit(X_train_encoded, y_train)
model_list.append(lasso_model)

### Linear regression

In [30]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train_encoded, y_train)
model_list.append(linear_model)

### ElasticNET

In [31]:
from sklearn.linear_model import ElasticNetCV
elastic_model = ElasticNetCV(cv=cv, n_jobs=-1, max_iter=10000)
elastic_model.fit(X_train_encoded, y_train)
model_list.append(elastic_model)

### Ridge

In [32]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=1e-10)
ridge_model.fit(X_train_encoded, y_train)
model_list.append(ridge_model)

## GradientBoosting

In [34]:
from sklearn.ensemble import GradientBoostingRegressor
ridge_model = GradientBoostingRegressor()
ridge_model.fit(X_train_encoded, y_train)
model_list.append(ridge_model)

## Make prediction

In [35]:
predict_list = []

for model in model_list:
    predict_list.append(model.predict(X_test_encoded))

## Plot of data

In [36]:
from matplotlib import pyplot as plt
from matplotlib import colors
import pandas as pd
import seaborn as sns
import numpy as np

ALL_COLORS = ["blue","orange","purple","green","gray"]

plt.figure(figsize=(400, 150))
sns.lineplot(x=range(len(y_test)), y=y_test.sort_values(), color='black', label='Actual')
plt.xlabel('Index')
plt.ylabel('Predicted Value')
plt.title('Line Plot of Predicted Values')

for i in range(0, len(model_list)):
    if i == 1:
        continue
    y = np.sort(predict_list[i])
    sns.lineplot(x=range(len(predict_list[i])), y=y, color=ALL_COLORS[i], label=type(model_list[i]).__name__)

plt.show()

## Check score of model

In [37]:
print("Scores: ")
for i, model in enumerate(model_list):
    print(f"\n* {type(model_list[i]).__name__}: {model.score(X_test_encoded, y_test)*100:.2f}%\t")

Scores: 

* LassoCV: 72.47%	

* LinearRegression: 62.78%	

* ElasticNetCV: 72.45%	

* Ridge: 62.78%	

* GradientBoostingRegressor: 96.78%	


## Mean errors

In [38]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

### Absolute

In [39]:
print("MAE (Mean Absolute Error): ")
for i, predict in enumerate(predict_list):
    mse = mean_absolute_error(y_true=y_test, y_pred=predict)
    print(f"\n* {type(model_list[i]).__name__}: {mse:.2f}\t")

MAE (Mean Absolute Error): 

* LassoCV: 133290.07	

* LinearRegression: 153297.70	

* ElasticNetCV: 133417.75	

* Ridge: 153297.70	

* GradientBoostingRegressor: 38801.12	


### Squared

In [40]:
print("MSE (Mean Squared Error): ")
for i, predict in enumerate(predict_list):
    print(f"\n* {type(model_list[i]).__name__}: {mean_squared_error(y_test, predict):.2f}\t")


MSE (Mean Squared Error): 

* LassoCV: 33794271721.09	

* LinearRegression: 45695056159.21	

* ElasticNetCV: 33823208794.89	

* Ridge: 45695056145.33	

* GradientBoostingRegressor: 3957071370.10	
